In [1]:
import numpy as np
import pandas as pd

import ecephys
import findlay2025a as f25a
import wisc_ecephys_tools as wet
from ecephys import wne

/home/gfindlay/projects/ece/gfys_workspace/.venv/lib/python3.13/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(


In [2]:
nbsh = wet.get_sglx_project("seahorse")
experiment = wet.rats.constants.SleepDeprivationExperiments.NOD
sortings = f25a.units.get_nod_sortings()

In [3]:
acg_metrics_file = nbsh.get_project_file("acg_metrics.pqt")
if acg_metrics_file.exists():
    acg_metrics = pd.read_parquet(acg_metrics_file)
else:
    acg_metrics = []
    for subject, probes in sortings:
        subject_metrics = pd.read_parquet(
            nbsh.get_experiment_subject_file(experiment, subject, "acg_metrics.pqt")
        )
        subject_metrics["subject"] = subject
        acg_metrics.append(subject_metrics)
    acg_metrics = pd.concat(acg_metrics)
    acg_metrics.to_parquet(acg_metrics_file)

In [ ]:
mua_threshold_kwargs = f25a.units.get_threshold_kwargs()["mua"]
mps_metrics_file = nbsh.get_project_file("mps_metrics.pqt")
if mps_metrics_file.exists():
    mps_metrics = pd.read_parquet(mps_metrics_file)
else:
    mps_metrics = []
    for subject, probes in sortings:
        mps = f25a.units.load_nod_multiprobe_sorting(subject, **mua_threshold_kwargs)
        mps_metrics.append(mps.properties)
    mps_metrics = pd.concat(mps_metrics)
    mps_metrics["acronym"] = mps_metrics["acronym"].apply(
        f25a.units.hippocampus_to_waxholm
    )
    mps_metrics = f25a.units.add_major_regions(mps_metrics, as_booleans=False)
    mps_metrics.to_parquet(mps_metrics_file)

In [5]:
df = pd.merge(
    acg_metrics,
    mps_metrics,
    on=["subject", "cluster_id"],
    suffixes=("_acg", None),
    how="left",
)
df.to_parquet(nbsh.get_project_file("aggregated_cell_metrics.pqt"))

In [ ]:
id_cols = ["subject", "experiment", "probe", "cluster_id"]
cluster_quality = (
    df[id_cols].drop_duplicates(ignore_index=True).assign(max_quality=np.nan)
)
for strictness in ["mua", "sua_permissive", "sua_moderate", "sua_conservative"]:
    threshold_kwargs = f25a.units.get_threshold_kwargs()[strictness]
    simple_filters, callable_filters = wne.siutils.get_quality_metric_filters(
        **threshold_kwargs
    )
    strict_df = ecephys.units.siutils.refine_clusters(
        df, simple_filters, callable_filters, include_nans=True
    )
    strict_df = strict_df[id_cols].drop_duplicates(ignore_index=True)
    for _, row in strict_df.iterrows():
        mask = (cluster_quality[id_cols] == row[id_cols]).all(axis=1)
        cluster_quality.loc[mask, "max_quality"] = strictness
cluster_quality.to_parquet(nbsh.get_project_file("cluster_quality.pqt"))

quality: {'good', 'mua', nan} excludes 0 clusters.
firing_rate: (0.5, inf) excludes 0 clusters.
0/19936 clusters excluded by jointly applying filters. 19936 remain.
quality: {'good', 'mua', nan} excludes 0 clusters.
firing_rate: (0.5, inf) excludes 0 clusters.
nn_isolation: (0.7, inf) excludes 1180 clusters.
amplitude_cutoff: (0.0, 0.499) excludes 9284 clusters.
Callable filter select_inviolate excludes 2464 clusters.
10532/19936 clusters excluded by jointly applying filters. 9404 remain.
quality: {'good', 'mua', nan} excludes 0 clusters.
firing_rate: (0.5, inf) excludes 0 clusters.
nn_isolation: (0.8, inf) excludes 2476 clusters.
amplitude_cutoff: (0.0, 0.499) excludes 9284 clusters.
Callable filter select_inviolate excludes 4220 clusters.
11704/19936 clusters excluded by jointly applying filters. 8232 remain.
quality: {'good', 'mua', nan} excludes 0 clusters.
firing_rate: (0.5, inf) excludes 0 clusters.
nn_isolation: (0.9, inf) excludes 6812 clusters.
amplitude_cutoff: (0.0, 0.3) exc